In [1]:
# 2020-01-09
# A. Pendleton
# Expression line plots for gene list generated from sample filtration steps

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys

#Plotting
import pandas as pd
import seaborn as sns
# import fastcluster

In [4]:
# For interactivity
from __future__ import print_function
import plotly
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) #Make it work in notebook mode
import plotly.graph_objs as go


init_notebook_mode(connected=True)



The below geneList would be a python list of gene identifiers that was propagated following sample filtration steps such as: 
    - Search by gene ID(s)
    - Search by homolog ID(s)
    - Search by functional annotations
    - Filtration by expression thresholds and/or patterns from the drop-down expression lists

## Inputs

This plot requires the following information:

- $ROOT/Genes_expression.txt
- list of genes to plot that was populated using any of the steps outlined in above cell. 

#### Define root directory

In [19]:
rootDir = '/depot/jwisecav/data/pendlea/coexpression_assessments/development/coexp_development/'
rootDir = '../'

#### Sample file

In [20]:
#Define the sample file
# File naming is static -- should always be the file name "samples.txt"
sampleFile = rootDir + 'samples.txt'

#### Global gene expression file

In [21]:
#Define expression file. 
# File naming is static -- should always be file name "Genes_expression.txt"
expFile = rootDir + 'Genes_expression.txt'


#### Gene list to plot

Here, I have an example of 82 genes that were defined in a module (N100M00016).

These genes can be found in the file below: 
    
    /depot/jwisecav/data/pendlea/coexpression_assessments/development/coexp_development/examples/example_geneList.txt


In [22]:
geneList = ["Sevir.9G282425", "Sevir.7G018910", "Sevir.5G044500", "Sevir.2G127900", "Sevir.5G129000", 
            "Sevir.2G354400", "Sevir.2G152600", "Sevir.7G023700", "Sevir.4G118900", "Sevir.8G081600", 
            "Sevir.5G364132", "Sevir.4G263301", "Sevir.9G099466", "Sevir.9G078500", "Sevir.2G263800", 
            "Sevir.8G218520", "Sevir.3G390900", "Sevir.9G165400", "Sevir.8G245140", "Sevir.8G087000", 
            "Sevir.9G576201", "Sevir.5G470750", "Sevir.1G060850", "Sevir.2G280500", "Sevir.9G268200", 
            "Sevir.4G143101", "Sevir.2G342800", "Sevir.1G019900", "Sevir.3G353500", "Sevir.7G189700", 
            "Sevir.4G118600", "Sevir.5G315900", "Sevir.1G110232", "Sevir.5G206100", "Sevir.3G353200", 
            "Sevir.4G093300", "Sevir.1G171100", "Sevir.2G305750", "Sevir.9G169700", "Sevir.1G020800", 
            "Sevir.5G194900", "Sevir.6G028050", "Sevir.2G003200", "Sevir.8G079100", "Sevir.4G293501", 
            "Sevir.7G022900", "Sevir.8G165900", "Sevir.9G092300", "Sevir.6G113200", "Sevir.4G160200", 
            "Sevir.5G037400", "Sevir.3G298200", "Sevir.2G437300", "Sevir.9G095700", "Sevir.1G265000", 
            "Sevir.5G418900", "Sevir.8G237700", "Sevir.2G177600", "Sevir.9G253801", "Sevir.6G183566", 
            "Sevir.8G259500", "Sevir.5G036550", "Sevir.7G279700", "Sevir.8G068450", "Sevir.9G576750", 
            "Sevir.9G248533", "Sevir.1G379100", "Sevir.8G222414", "Sevir.6G039200", "Sevir.6G075100", 
            "Sevir.3G331800", "Sevir.4G164200", "Sevir.9G347733", "Sevir.2G396750", "Sevir.3G116100", 
            "Sevir.7G061100", "Sevir.2G307100", "Sevir.5G087400", "Sevir.7G005300", "Sevir.2G026100", 
            "Sevir.9G072900"]
print('%i genes in geneList' % len(geneList))

81 genes in geneList


We have set the maximum number of genes to view in our plots to 100. The below cell will redefine which genes we plot from the above geneList in case the geneList is longer than 100.

In [23]:
#What is the maximum number of genes to plot?
maxGenesToPlot = min(100, len(geneList)) #Either 100 or the length of the total gene list

#If the length of genes in geneList is

#Get the list of genes we'll be plotting
genesToPlot = geneList[0:maxGenesToPlot]

print('%i genes in geneList' % len(geneList))     
print('%i genes to plot' % len(genesToPlot))


81 genes in geneList
81 genes to plot


___________

### Store the sample information
##### Link the sample IDs to conditions and experiments

In [24]:
#Dictionary to store the sample information
# the dictionary should have the following structure:
#.      sampleDict[sampleID] = condition

sampleDict = {} 
conditionDict = {}
experiments = []

lineCount = 0# Track the line count while parsing sample file

for line in open(sampleFile, 'r'): #Parse each data file line
    line=line.rstrip().split('\t') #strip and split this tab-delimited file
    
    lineCount += 1 #add one to line count
    
    #Ignore header line
    if lineCount == 1:
        continue
    
    sampleID = line[0]
    condition = line[1] 
    experiment = line[2]
    
    sampleDict[sampleID] = [condition, experiment] #Track how samples are linked to conditions and experiments
    conditionDict[condition] = experiment
    experiments.append(experiment)
    
#Also append to experiments 
print('Stored sample-condition links for %i samples' % len(sampleDict.keys()))
print('    from %i conditions' % len(conditionDict.keys()))

experiments = np.unique(experiments)
print('    and %i experimental groups' % len(experiments))


Stored sample-condition links for 268 samples
    from 52 conditions
    and 7 experimental groups


___________

### Store gene expression data for genes in geneList

In [25]:

expDict = {}
samples = [] #Store the samples, in order defined by header line
conditions = []
sampleExpDict = {} #Store expression for ALL samples in one dictionary

lineCount = 0 #track line count

for line in open(expFile, 'r'):
    line=line.rstrip().split('\t') #split by tab -- is a tab-delimited file 
    lineCount += 1
    
    #Sample identifiers stored in header line. Will use samples as keys for expDict
    if lineCount == 1:
        for i in range(1,len(line)): #Sample names start at column 2 (index 1), column 1 == gene
            sampleID = line[i]
            samples.append(sampleID)
            
            #Determine condition based on the sample ID by searching sampleDict
            condition = sampleDict[sampleID][0]
            conditions.append(condition)
            
        print('%i samples stored in samples list' % len(samples))
        conditions = np.unique(conditions)
        print('    from %i conditions' % len(conditions))
        continue
        
    #Gene ID always in column 1
    geneID = line[0]
    
    #If the gene is not in the geneList list, continue
    if geneID not in geneList:
        continue
    
    #Start populating dictionary with the following structure
    #.    expDict[geneID][sampleID] = [[sampleID,expressionValue],[sampleID,expressionValue],[sampleID,expressionValue]...]
    expDict[geneID] = {}
    for condition in conditionDict.keys(): #Set up cleared lists to track expression levels per condition
        experiment = conditionDict[condition]
        if experiment not in expDict[geneID].keys():
            expDict[geneID][experiment] = {}
        expDict[geneID][experiment][condition] = []
    
    #Create empty key for samples in sampleExpDict
    sampleExpDict[geneID] = {}
    
    #Create new key per sampleID
    for sampleID in samples:
        #What column is the sample in samples?
        SampleColumnNumber = samples.index(sampleID)
        
        #What condition does this sample belong to?
        #Determine the condition the sample belongs to from sampleDict dictionary
        condition = sampleDict[sampleID][0]
        experiment = sampleDict[sampleID][1]
            
        #Now extract expression level from same column that matches the sample
        expressionValue = float(line[SampleColumnNumber+1])
        
        #Store in dictionary
        expDict[geneID][experiment][condition].append(expressionValue)

        #Populate dictionary to store the samples' expression
        sampleExpDict[geneID][sampleID] = expressionValue

print('Experimental expression values for %i genes stored' % len(expDict.keys()))


268 samples stored in samples list
    from 52 conditions
Experimental expression values for 81 genes stored


#### Calculate the average expression value, per gene, per condition

In [26]:
meanExpressionDict = {}

for geneID in expDict.keys():
    meanExpressionDict[geneID] = {} #Define the average dictionary for this gene
    
    for condition in conditionDict.keys():  
        experiment = conditionDict[condition] #get this condition's experimental group it belongs to
        #Create experiment key
        if experiment not in meanExpressionDict[geneID].keys():
            meanExpressionDict[geneID][experiment] = {}
            
        #Get the array of total expression values for this gene under the given condition
        allExpressionValues = expDict[geneID][experiment][condition]
        
        #Now calculate the mean, minimum, and maximum expression
        meanExpression = np.mean(allExpressionValues) 
        minExpression = min(allExpressionValues)
        maxExpression = max(allExpressionValues)
        
        meanExpressionDict[geneID][experiment][condition] = [meanExpression, minExpression, maxExpression] #store the expression value
print('Average expression levels stored for %i genes' % len(meanExpressionDict.keys()))   

Average expression levels stored for 81 genes


___________

## Line plot

Here, we will draw line plots that will show the expression levels per gene (with standard error bars) across all conditions. 

In [27]:
def get_plot_colors(genesToPlot):
    NUM_COLORS = len(genesToPlot) #how many colors our plot will have depends on how many genes
    
    #The number of line styles we need also depends on the number of genes we have
    # if we dont have a lot, we will only use solid lines. While more complexity is needed
    # as gene count increases, so we can distinguish each gene
    if NUM_COLORS < 20:
        LINE_STYLES=['solid']
    if NUM_COLORS >= 20 and NUM_COLORS <= 50:
        #LINE_STYLES = ['solid', 'dashed', 'dashdot']
        LINE_STYLES = ['solid', 'dot']
    if NUM_COLORS >= 50:
        #LINE_STYLES = ['solid', 'dashed', 'dashdot', 'dotted']
        LINE_STYLES = ['solid', 'dot', 'dash']
    NUM_STYLES = len(LINE_STYLES)

    #return the following plotting variables
    return NUM_COLORS, LINE_STYLES, NUM_STYLES

#### Colors
Below are 500 random colors for plotting

In [28]:
colors =  [
"#63b598", "#ce7d78", "#ea9e70", "#a48a9e", "#c6e1e8", "#648177" ,"#0d5ac1" ,
"#f205e6" ,"#1c0365" ,"#14a9ad" ,"#4ca2f9" ,"#a4e43f" ,"#d298e2" ,"#6119d0",
"#d2737d" ,"#c0a43c" ,"#f2510e" ,"#651be6" ,"#79806e" ,"#61da5e" ,"#cd2f00" ,
"#9348af" ,"#01ac53" ,"#c5a4fb" ,"#996635","#b11573" ,"#4bb473" ,"#75d89e" ,
"#2f3f94" ,"#2f7b99" ,"#da967d" ,"#34891f" ,"#b0d87b" ,"#ca4751" ,"#7e50a8" ,
"#c4d647" ,"#e0eeb8" ,"#11dec1" ,"#289812" ,"#566ca0" ,"#ffdbe1" ,"#2f1179" ,
"#935b6d" ,"#916988" ,"#513d98" ,"#aead3a", "#9e6d71", "#4b5bdc", "#0cd36d",
"#250662", "#cb5bea", "#228916", "#ac3e1b", "#df514a", "#539397", "#880977",
"#f697c1", "#ba96ce", "#679c9d", "#c6c42c", "#5d2c52", "#48b41b", "#e1cf3b",
"#5be4f0", "#57c4d8", "#a4d17a", "#225b8e", "#be608b", "#96b00c", "#088baf",
"#f158bf", "#e145ba", "#ee91e3", "#05d371", "#5426e0", "#4834d0", "#802234",
"#6749e8", "#0971f0", "#8fb413", "#b2b4f0", "#c3c89d", "#c9a941", "#41d158",
"#fb21a3", "#51aed9", "#5bb32d", "#807fbc", "#21538e", "#89d534", "#d36647",
"#7fb411", "#0023b8", "#3b8c2a", "#986b53", "#f50422", "#983f7a", "#ea24a3",
"#79352c", "#521250", "#c79ed2", "#d6dd92", "#e33e52", "#b2be57", "#fa06ec",
"#1bb699", "#6b2e5f", "#64820f", "#59fa98", "#21538e", "#89d534", "#d36647",
"#7fb411", "#0023b8", "#3b8c2a", "#986b53", "#f50422", "#983f7a", "#ea24a3",
"#79352c", "#521250", "#c79ed2", "#d6dd92", "#e33e52", "#b2be57", "#fa06ec",
"#1bb699", "#6b2e5f", "#64820f", "#9cb64a", "#996c48", "#9ab9b7",
"#06e052", "#e3a481", "#0eb621", "#fc458e", "#b2db15", "#aa226d", "#792ed8",
"#73872a", "#520d3a", "#cefcb8", "#a5b3d9", "#7d1d85", "#c4fd57", "#f1ae16",
"#8fe22a", "#ef6e3c", "#243eeb", "#f870de", "#dd93fd", "#3f8473", "#e7dbce",
"#421f79", "#7a3d93", "#635f6d", "#93f2d7", "#9b5c2a", "#15b9ee", "#0f5997",
"#409188", "#911e20", "#1350ce", "#10e5b1", "#fff4d7", "#cb2582", "#ce00be",
"#32d5d6", "#7721ea", "#608572", "#c79bc2", "#00f87c", "#77772a", "#6995ba",
"#fc6b57", "#f07815", "#8fd883", "#060e27", "#96e591", "#21d52e", "#d00043",
"#b47162", "#1ec227", "#4f0f6f", "#1d1d58", "#947002", "#bde052", "#e08c56",
"#28fcfd", "#a0882c", "#36486a", "#d02e29", "#1ae6db", "#3e464c", "#a84a8f",
"#911e7e", "#3f16d9", "#0f525f", "#ac7c0a", "#b4c086", "#c9d730", "#30cc49",
"#3d6751", "#fb4c03", "#640fc1", "#62c03e", "#d3493a", "#88aa0b", "#406df9",
"#615af0", "#8f5c71", "#2a3434", "#4a543f", "#79bca0", "#a8b8d4", "#00efd4",
"#7ad236", "#7260d8", "#1deaa7", "#06f43a", "#823c59", "#e3d94c", "#dc1c06",
"#f53b2a", "#b46238", "#2dfff6", "#a82b89", "#1a8011", "#436a9f", "#1a806a",
"#4cf09d", "#c188a2", "#67eb4b", "#b308d3", "#fc7e41", "#af3101", "#e8ca24",
"#71b1f4", "#a2f8a5", "#e23dd0", "#d3486d", "#00f7f9", "#474893", "#3cec35",
"#1c65cb", "#5d1d0c", "#2d7d2a", "#ff3420", "#5cdd87", "#a259a4", "#e4ac44",
"#1bede6", "#8798a4", "#d7790f", "#b2c24f", "#de73c2", "#d70a9c", "#25b67c",
"#88e9b8", "#c2b0e2", "#86e98f", "#ae90e2", "#1a806b", "#436a9e", "#0ec0ff",
"#f812b3", "#b17fc9", "#8d6c2f", "#d3277a", "#2ca1ae", "#9685eb", "#8a96c6",
"#dba2e6", "#76fc1b", "#608fa4", "#20f6ba", "#07d7f6", "#dce77a", "#77ecca"]

#### Drawing plot

In [29]:

#Determine plotting color schema
NUM_COLORS, LINE_STYLES, NUM_STYLES = get_plot_colors(genesToPlot)
cm = plt.get_cmap('jet')

#Print by experiment
for experiment in experiments:    
    labels=[] #this is to store your conditions, if you want them visible along the x axis, the plt.legend will use this 
    
    geneCount = 0

    trackGlobalY = [] #Keep track of the max expression (yvalue) observed so we can dynanmically change y axis
    
    globalData = []
    
    data = []
    
    #this will draw the line for each gene as the loop goes on
    for geneID in genesToPlot:
        geneCount+=1
        
        x, y = [], [] #Clear the x,y variables for plotting line

        #Get the color based on how many genes have been processed so far
        COLOR = cm(geneCount/NUM_STYLES*float(NUM_STYLES)/NUM_COLORS)
        LINE_STYLE = LINE_STYLES[geneCount%NUM_STYLES]
        
        #go through each condition, and then plot the average expression level for that condition
        for condition in meanExpressionDict[geneID][experiment].keys():            
            x.append(condition) # Add the condition name so it willl show up on the x axis

            #Now you want to append the average expression value
            averageExpressionValue = float(meanExpressionDict[geneID][experiment][condition][0])
            y.append(averageExpressionValue) #append the value
            trackGlobalY.append(averageExpressionValue) #Track all mean y variables. This way you can find the maximum and dynamically change your y axis


        #Draw the line
        #Color equals the number of genes observed and selected from our random gene array colors [above]
        linePlotData = go.Scatter(x = x, y = y, mode = 'lines+markers', name = geneID, line = dict(width = 4, dash=LINE_STYLE, color=colors[geneCount]))
        data.append(linePlotData) #add this line's information 
            
    
    # Edit the layout - including title and axis labels
    #Change grid color == xaxis/yaxis gridcolor 
    #Change background color == plot_bgcolor
    #Change title == title
    layout = dict(title = 'Average Gene Expression - %s' % experiment,
              yaxis = dict(title = 'Expression', showgrid=True, gridcolor='whitesmoke'),
              xaxis = dict(title = 'Experimental Conditions', showgrid=True, gridcolor='whitesmoke'),
              plot_bgcolor='white', dragmode='select')
    
    #Define the figure's visual layout and data, plotly wise
    fig = dict(data=data, layout=layout)
    
    #PLOT!
    iplot(fig)
    


